In [5]:
import pandas as pd
import numpy as np

all_data = pd.read_stata('long_study_school.dta', convert_categoricals=False)
all_data.head()

,CASENUM,COHORT,SCHOOLID,STRATA,ASCICLS,ASCITCH,ASTSEX,AMTHCLS,AMTHTCH,AMTSEX,...,PEDUC3,MEDSRCE,FEDSRCE,MOTHOCC,FATHOCC,POCI,FOCCSRCE,MOCCSRCE,MOTHSEI,FATHSEI
0,1001,1,309,6,-95,-95,-95,-95,-95,-95,...,1,2,3,395,785,0,5,4,34,15
1,1002,2,132,11,132032,13203,1,132061,13206,2,...,4,7,7,-99,-99,-99,-99,-99,-99,-99
2,1003,1,309,6,-95,-95,-95,-95,-95,-95,...,1,3,2,988,535,0,4,5,-98,27
3,1004,2,126,8,126026,12602,2,126101,12610,2,...,1,8,8,270,331,0,11,11,55,53
4,1005,2,133,11,-99,-99,-99,133024,13302,1,...,4,3,7,65,471,0,2,3,87,32


Looking at the results of the head() function, this dataset has 11,904 columns, which is far more than will be useful for a single exploratory data analysis project. In order to get a better idea as to what data I may look at, I skimmed the associated codebook, which gives a description for each column name.

I decided to explore the hours of homework completed by 10th graders. I plan to create a sum of all the homework hours per week across all subjects, adding 0 if there is a response of -99 (which according to the codebook, indicates that they did not take a respective course during that term) and recording if the data is blank (-98) or they did not participate in that question (-95). From this, I can look to see if there are any common outcomes predicted by the number of hours of homework each week. 
I would like to look at 10th graders specifically, as it is the last year in high school before college and major decisions are made, considering that most college applications are due in the fall of junior year. There may be information in this dataset which allows me to make predictions about college and career success based on student attitudes during this year.

I would be interested in looking at 

In [10]:
'''
This code filters the table to only include columns which correspond to the appropriate codes for the number of hours of homework 
reported by 10th graders.
The first letter is either G or H, meaning that this includes the reponses from the 10th graders in the fall (G) and in the spring (H).
The next letter must be A after the first, because the 'A' group asks questions directly related to schoolwork.
Next, there is a 3 character code related to the subject (for example, MTH for Math and COM for Computer). This is matched as any 
three characters in the regular expression.
After this three character subject marker, there may or may not be a digit appearing afterwards. This is present when there are more 
than one of a subject that a student can be enrolled in during that semester, and in that case the first class will have a 1 added 
and the second a 2. If the regular expression does not look for this number, any subjects with more than one option would be excluded.
Finally, the column code needs to end in the letter J, as this is the code in section A that asks how many hours of homework a week
that students has for that subject.

The CASENUM column should also be included in order to maintain which student the questions correspond to.

Note, the filter method does not remove the columns which don't make it past the filter, so this is stored as a new dataframe, tenth_hours.
'''
tenth_hours = all_data.filter(regex=("^([GH]A.{3}[12]?J)|(CASENUM)$"))
tenth_hours.head()

,CASENUM,GAMTH1J,GAMTH2J,GASCI1J,GASCI2J,GAENG1J,GAENG2J,GASSTJ,GACOMJ,GAFORJ,...,HASCI1J,HASCI2J,HAENG1J,HAENG2J,HASSTJ,HACOMJ,HAFORJ,HAARTJ,HAMUSJ,HAVOCJ
0,1001,1,-99,3,-99,1,-99,-99,-99,-99,...,2,-99,1,-99,2,-99,-99,-99,-99,0
1,1002,5,-99,3,-99,3,-99,-99,-99,-99,...,5,-99,1,-99,-99,-99,-99,0,-99,-99
2,1003,3,-99,4,-99,3,-99,-99,-99,4,...,4,-99,4,-99,-99,-99,3,-98,-99,-99
3,1004,-95,-95,-95,-95,-95,-95,-95,-95,-95,...,-95,-95,-95,-95,-95,-95,-95,-95,-95,-95
4,1005,1,-99,1,-99,1,-99,-99,-99,1,...,1,-99,0,-99,-99,-99,1,-99,1,-99


By using the head() function, we can confirm that we did retrieve the desired columns. Another option would be to print the columns method of the dataframe. I also can double check that there is an appropriate number of columns; 24. Double checking with the codebook, this is indeed the number of codes that are questions asked to 10th graders about the number of hours per week spent on homework.

Although this filtered dataset contains considerably less data than the original dataset, it is still not easily interpreatble. 
Because there are already so many columns in the filtered dataset, creating a new dataframe which contains the Case number and for now, the number of hours total spent per week across all subjects doing homework, the number of classes a student was taking, and the number of questions a student skipped or did not participate in.
Including the information about why a question was not answered could potentially give insight into the accuracy of the conclusions ofthe data, and it may also allow a case to be excluded if they did not participate in any of the questions (sum is 24 as there are 24 codes observed).
This new dataframe could also be used later to store more statistics about a student. The number of classes could help determine an average time per class, or point to a student who has less hours because the given subjects do not describe their actual schedule well.

In [28]:
statistics = pd.DataFrame(columns=['CASENUM', 'HOURSTOTAL', 'SKIPS', 'NONPART', 'NUMENROLLED'])


# Getting the column names for the full tenth grade datset.
# The values at CASENUM are not answers to a question and is removed from this list
columns = list(tenth_hours)
columns.remove('CASENUM')

for index, row in tenth_hours.iterrows():
    total_hours = 0
    skips = 0
    nonpart = 0
    num_enrolled = 0

    # iterate over each column name in the dataframe
    for code in columns:
        if row[code] == -98:
            # response is blank (no response)
            skips += 1
            
        elif row[code] == -95:
            # student did not participate in this question
            nonpart += 1

        elif row[code] != -99:
            num_enrolled += 1
            total_hours += row[code]
        
    statistics.loc[len(statistics.index)] = [row['CASENUM'], total_hours, skips, nonpart, num_enrolled]

statistics.head()

,CASENUM,HOURSTOTAL,SKIPS,NONPART,NUMENROLLED
0,1001,10,0,0,8
1,1002,19,0,0,8
2,1003,28,1,0,9
3,1004,0,0,24,0
4,1005,8,0,0,10


Next, the cases that did not participate in any questions should be removed since they do not record any of the data we are looking for.

In [ ]:
statistics = statistics[statistics.NONPART != 24]

statistics.head()